In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from datetime import datetime,date,timedelta

In [2]:
rds_connection_string = "root:12345678@127.0.0.1/fires"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')


In [3]:
wildfire_df = pd.read_sql("select * from wildfire_data",con=engine)
wildfire_df.head()

/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,index,discovery_date,containment_date,size,class,fire_year,fire_name
0,0,2005-02-02,2005-02-02,0.10,A,2005,FOUNTAIN
1,1,2004-05-12,2004-05-12,0.25,A,2004,PIGEON
2,2,2004-05-31,2004-05-31,0.10,A,2004,SLACK
3,3,2004-06-28,2004-07-03,0.10,A,2004,DEER
4,4,2004-06-28,2004-07-03,0.10,A,2004,STEVENOT


In [4]:
wildfire_complete = wildfire_df.dropna()
#fill in missing values using average by each group
wildfire_complete["duration"] = (wildfire_complete["containment_date"] - wildfire_complete["discovery_date"]).dt.days
duration_class = wildfire_complete.groupby("class").agg({"duration":"mean"})
duration_class_dict = duration_class.to_dict()["duration"]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
duration_class_dict

{'A': 0.5788469890666013,
 'B': 1.1438816490451602,
 'C': 2.139679255043973,
 'D': 6.192269573835481,
 'E': 6.223479490806223,
 'F': 14.183823529411764,
 'G': 27.64306784660767}

In [6]:
wildfire_df[wildfire_df["containment_date"].isnull()].iloc[0:10,]

,index,discovery_date,containment_date,size,class,fire_year,fire_name
821,821,2005-12-05,NaT,0.1,A,2005,PAINTED
2035,2035,2006-08-23,NaT,7.0,B,2006,NEILSON
2203,2203,2006-09-27,NaT,5.0,B,2006,WHEELER
2235,2235,2006-06-26,NaT,1247.0,F,2006,HARVEY
2294,2294,2006-07-23,NaT,5792.0,G,2006,TITUS
2524,2524,2006-08-22,NaT,0.1,A,2006,BEAR
2599,2599,2006-09-03,NaT,178.0,D,2006,SAND FLAT
2600,2600,2006-07-21,NaT,0.1,A,2006,DEER
2924,2924,2006-09-29,NaT,0.5,B,2006,BLUE
3816,3816,2007-09-24,NaT,0.1,A,2007,WILLIS


In [ ]:
for index, row in wildfire_df.iterrows():
    if pd.isnull(row["containment_date"]):
        this_class = row["class"]
        wildfire_df.loc[index,"containment_date"] = row["discovery_date"] + pd.Timedelta(str(duration_class_dict[this_class]) + " days")

In [ ]:
wildfire_df[wildfire_df["containment_date"].isnull()]

In [ ]:
wildfire_df.to_sql(name="cleaned_wildfires", con=engine, if_exists="append", index=True)

In [ ]:
dates_with_fire = []
fire_acres = []
for index, row in wildfire_df.iterrows():
    duration = row["containment_date"] - row["discovery_date"]
    current_dates_with_fire = [row["discovery_date"] + timedelta(days=x) for x in range(duration.days + 1) ]
    for x in current_dates_with_fire:
        dates_with_fire.append(x)
        fire_acres.append(row["size"])

In [ ]:
type(dates_with_fire)

In [ ]:
pd.DataFrame.from_records(dates_with_fire, fire_acres)

In [ ]:
pd.DataFrame.from_records(sales, columns=labels)

In [ ]:
#truncate dates
truncated_dates_with_fire = [x.replace(hour = 0,minute = 0, second = 0,microsecond = 0 , nanosecond = 0) for x in dates_with_fire]
unique_dates_with_fire = list(set(truncated_dates_with_fire))

In [ ]:
air_quality_df = pd.read_sql("select * from aqi_data",con=engine)
air_quality_df["total_size"] = 0
air_quality_df.head()

In [ ]:
wildfire_df = wildfire_df.iloc[0:100]

In [ ]:
for index, row in wildfire_df.iterrows():
    fire_size = row["size"]
    duration = (row["containment_date"] - row["discovery_date"]).days
    date = wildfire_df["discovery_date"]
    for i in range (1, duration):
        row_index = air_quality_df[air_quality_df["date"] == date
        print(air_quality_df.loc[:,"total_size"].iloc[row_index,])                           
     #   (air_quality_df.iloc[row_index])["total_size"] = (air_quality_df.iloc[row_index])["total_size"] + fire_size    
        date = date + 1
        
#    while date <= wildfire_df["containment_date"]
 #       air_quality_df.loc[:,"total_size"].iloc[row_index,] = air_quality_df.loc[:,"total_size"].iloc[row_index,] + fire_size
  #      date = date + 1

In [ ]:
for index, row in wildfire_df.iloc[0:100,].iterrows():
    fire_size = row["size"]
    duration = row["containment_date"] - row["discovery_date"]
    current_dates_with_fire = [row["discovery_date"] + timedelta(days=x) for x in range(duration.days + 1)]
    for x in current_dates_with_fire:
        row_index = air_quality_df[air_quality_df["date"] == x].index.values[0]
         = air_quality_df.loc[:,"total_size"].iloc[row_index,] + fire_size
        #air_quality_df.loc[,"total_size"].iloc[row_index,] = air_quality_df[air_quality_df["date"] == x]["total_size"] + fire_size
        #air_quality_df[air_quality_df["date"] == x]["total_size"] += fire_size

In [ ]:
air_quality_df.head()

In [ ]:
air_quality_df[(air_quality_df["total_size"] != 0)]

In [ ]:
df[(df['A']>0) & (df['B']>0) & (df['C']>0)].count()

In [ ]:
air_quality_df.loc["total_size" != 0]